# Introduction to [Plotly](https://plotly.com/)

Plotly is a versatile interactive plotting package that can be used with Python and Javascript and also through an online editor (without the need for coding). 

## Why/When to use Plotly (my 2 cents)

If you already know Python and you don't really want to learn another coding language, but you do want to create interactive figures (e.g., within a Jupyter notebook and/or for use on a website), you should look into Plotly.  

In particular, [Plotly express](https://plotly.com/python/plotly-express/) is a fantastic tool for generating quick interactive figures without much code.  Plotly express covers a good amount of ground, and you may be able to do all/most your work within Plotly express, depending on your specific needs.  In this workshop, I'll show you Plotly express, but then move beyond it for the majority of the content.  

Though you can do a lot with Plotly, ity definitely has limitations (some of which we'll see in this workshop). Also, as with all of thee ready-made interactive plot solutions (e.g., [Bokeh](https://docs.bokeh.org/en/latest/), [Altair](https://altair-viz.github.io/), [Glue](https://glueviz.org/), etc.), Plotly has a specific look, which can only be tweaked to a certain extent.  If you like the look well enough and you don't mind the limitations, then it's a good choice. 

##  In this tutorial... 

We will explore the basics of the Python version, using COVID-19 data from the following sources:

- COVID-19 data from the WHO: https://covid19.who.int/info/ 
- GDP Data from the World Bank: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD

I will make two plots, one comparing COVID-19 data to GDPs and another showing COVID-19 data as a function of time.

## Installation

I recommend installing Python using [Anaconda](https://www.anaconda.com/products/individual).  Then you can create and activate a new environment for this workshop by typing the following commands into your (bash) terminal.

```
$ conda create -n plotly-env python=3.9 jupyter pandas plotly statsmodels
$ conda activate plotly-env
```

## Import the relevant packages that we will use.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

## 1. Create a plot showing COVID-19 and GDP data.

### 1.1. Read in the data.

I will join multiple data tables together, using the *pandas* package so that I have one DataFrame containing all values for a given country.

In [ ]:
# Current cumulative COVID-19 data from the WHO. 
# dfCT = pd.read_csv('data/WHO-COVID/WHO-COVID-19-global-table-data.csv') # in case the WHO server goes down
dfCT = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-table-data.csv', index_col=False)
dfCT

In [ ]:
# Current vaccination data from the WHO
# dfV = pd.read_csv('data/WHO-COVID/vaccination-data.csv') # in case the WHO server goes down
dfV = pd.read_csv('https://covid19.who.int/who-data/vaccination-data.csv')
dfV

In [ ]:
# Vaccination metadata from the WHO; this file contains the start dates (and end dates) for vaccines for each country. 
# dfVM = pd.read_csv('data/WHO-COVID/vaccination-metadata.csv') # in case the WHO server goes down
dfVM = pd.read_csv('https://covid19.who.int/who-data/vaccination-metadata.csv')

# drop columns without a start date
dfVM.dropna(subset = ['START_DATE'], inplace = True)

# convert the date columns to datetime objects for easier plotting and manipulation later on
dfVM['AUTHORIZATION_DATE'] = pd.to_datetime(dfVM['AUTHORIZATION_DATE'])
dfVM['START_DATE'] = pd.to_datetime(dfVM['START_DATE'])
dfVM['END_DATE'] = pd.to_datetime(dfVM['END_DATE'])

# I will simplify this table to just take the earliest start date for a given country
# sort by the start date and country code
dfVM.sort_values(['START_DATE', 'ISO3'], ascending = (True, True), inplace = True)
# take only the first entry for a given country
dfVM.drop_duplicates(subset = 'ISO3', keep = 'first', inplace = True)

dfVM

In [ ]:
# GDP data from the World Bank (the first three rows do not contain data)
# I don't think there's a direct link to this data on their server (but I didn't look very hard)
dfM = pd.read_csv('data/WorldBank/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_3469429.csv', skiprows = 3)
dfM

In [ ]:
# Join these 4 tables so that I have one DataFrame with all values for a given country.
# I will start by joining the two vaccination data tables.
dfJ1 = dfV.join(dfVM.set_index('ISO3'), on = 'ISO3', how = 'left', rsuffix = '_meta')

# Next I will join this with the COVID-19 data table.
# First rename this column in the COVID-19 data so that it is the same as the vaccine data.  Then I will join on that column.
dfCT.rename(columns = {'Name':'COUNTRY'}, inplace = True)
dfJ2 = dfJ1.join(dfCT.set_index('COUNTRY'), on = 'COUNTRY', how = 'left')

# Finally, I will join in the GDP data from the World Bank.
# I will rename a column in the World Bank data to match a column in the joined data above.
dfM.rename(columns = {'Country Code':'ISO3'}, inplace = True)
dfJoinedCOVID = dfJ2.join(dfM.set_index('ISO3'), on = 'ISO3', how = 'left')

dfJoinedCOVID

### 1.2. Create a couple simple Plotly figures using [Plotly express](https://plotly.com/python/plotly-express/).

Plotly express is a simplified version of the Plotly interface for Python that allows users to create many types of Plotly figures with single lines of code.  This greatly simplifies the workflow for some kinds of Plotly figures.  We will start with Plotly express (and for some of your use cases, that may be enough), but we will move on to full blown Plotly for the rest of this workshop.

In this plot, I will show total vaccinations vs. GDP with the point size scaled by the total cumulative COVID-19 cases.  

In [ ]:
# Note: We imported plotly.express as px
# I will create a scatter plot using the DataFrame I created above, define the keys for the x and y data, 
#   plot the y axis in the log, and also size each data point by the number of COVID-19 cases.
# A nice part of plotly express is that you can add a trend line very easily.
fig = px.scatter(dfJoinedCOVID, x = 'TOTAL_VACCINATIONS_PER100', y = '2020', log_y = True, 
    size = np.nan_to_num(np.clip(dfJoinedCOVID['Cases - cumulative total per 100000 population']/500., 5, 100)),
    trendline = 'ols', trendline_options = dict(log_y = True)
)
fig.show()

Lets also look at the first vaccination start date vs GDP with the size based on to the total vaccionations.  In this example, I will also modify the hover and axis attributes.

In [ ]:
# The command is similar to that from the previous cell, but here I'm also defining the data shown on hover in the tooltips.
# (It's not quite as easy to add a trendline here when plotting dates, though it is possible.)
fig = px.scatter(dfJoinedCOVID, x = 'START_DATE', y = '2020', log_y = True, 
    size = dfJoinedCOVID['TOTAL_VACCINATIONS_PER100'],
    hover_name = 'COUNTRY', 
    hover_data = ['2020', 
      'START_DATE', 
      'TOTAL_VACCINATIONS_PER100',
      'Cases - cumulative total per 100000 population'
    ]
)

# a few manipulations to the axes 
fig.update_xaxes(title = 'Vaccine Start Date', range = [np.datetime64('2020-07-01'), np.datetime64('2021-07-01')])
fig.update_yaxes(title = '2020 GDP (USD)')

fig.show()

### *Exercise 1: Create your own plot using Plotly express.*

Use the data we read in above (or your own data).  You can start with one of the commands above or choose a different style of plot.  Whichever format you use, choose different columns to plot than above.  Try to also add a new option to the command to change the plot.  

Hint: Go to the [Plotly express homepage](https://plotly.com/python/plotly-express/), and click on a link to see many examples (e.g., [here's the page for the scatter plot](https://plotly.com/python/line-and-scatter/))

In [ ]:
# Create a plot using Plotly express




### 1.4. Create the plot using the standard Plotly [Graph Object](https://plotly.com/python/graph-objects/).

For the remainder of the workshop we will use Graph Objects for our Plotly figures.  One motivation here is so that I can create multiple panels in one figure, which can be downloaded to an html file.  (Plotly express will only make an individual figure, and cannot to subplots.) 

First you create a <b>"trace"</b>, which holds the data.  There are many kinds of traces available in Plotly. (e.g., bar, scatter, etc.).  For this example, we will use a scatter trace.  (Interestingly, the scatter trace object also includes line traces, accessed by changed the "mode" key.  I will show the line version later on.)

Then you create a figure and add the trace to that figure.  A single figure can have multiple traces.

In [ ]:
# Note: We imported the plotly.graph_objects as go.
# create the trace and set various parameters
trace1 = go.Scatter(x = dfJoinedCOVID['TOTAL_VACCINATIONS_PER100'], y = dfJoinedCOVID['2020'], # x and y values for the plot
    mode = 'markers', # setting mode to markers produces a typical scatter plot
    showlegend = False, # since I will only have one trace, I don't need a legend
    name = 'COVID Vaccines', # name for the legend and tooltips (this is not strictly necessary here)
    # set various parameters for the markers in the following dict, e.g., color, opacity, size, outline, etc.
    marker = dict( 
        color = 'rgba(0, 0, 0, 0.2)',
        opacity = 1,
        size = np.nan_to_num(np.clip(dfJoinedCOVID['Cases - cumulative total per 100000 population']/500., 5, 100)),
        line = dict(
            color = 'rgba(0, 0, 0, 1)',
            width = 1
        ),
    ),
    # set a template for the tooltips below.  
    # hovertemplate can accept the x and y data and additional "text" as defined by a separate input
    hovertemplate = '%{text}' + 
        'Total Vaccinations / 100 people: %{x}<br><extra></extra>' +
        'GDP: $%{y}<br>',
    # additional text to add to the hovertemplate.  This needs to be a list with the same length and the x and y data.
    text = ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Vaccine start date: {}<br>'.format(x1, x2, x3) 
        for (x1, x2, x3) in zip(dfJoinedCOVID['COUNTRY'], 
            dfJoinedCOVID['Cases - cumulative total per 100000 population'], 
            dfJoinedCOVID['START_DATE'].dt.strftime('%b %Y'))
    ],
    # style the tooltip as desired                
    hoverlabel = dict(
        bgcolor = 'white',
    )
)

# Add a trendline
# I will use scipy.stats.linregress (and fit to the log of the GDP)
dfFit1 = dfJoinedCOVID.dropna(subset = ['TOTAL_VACCINATIONS_PER100', '2020'])
slope1, intercept1, r1, p1, se1 = scipy.stats.linregress(dfFit1['TOTAL_VACCINATIONS_PER100'], np.log10(dfFit1['2020']))
xx1 = np.linspace(0, 300, 100)
yy1 = 10.**(slope1*xx1 + intercept1)
trace1F = go.Scatter(x = xx1, y = yy1, 
    mode = 'lines', # Set the mode the lines (rather than markers) to show a line.
    opacity = 1, 
    marker_color = 'black',
    showlegend = False,
    hoverinfo='skip' # Don't show anything on hover.  (We could show the trendline info, but I'll leave that out for now.)
)

# create the figure
fig = go.Figure()

# add the trace and update a few parameters for the axes
fig.add_trace(trace1)
fig.add_trace(trace1F)
fig.update_xaxes(title = 'Total Vaccionations Per 100 People', range=[0,300])
fig.update_yaxes(title = 'GDP (USD)', type = 'log')

fig.show()

### *Exercise 2: Create your own plot using Plotly Graph Object(s).*

Use the data we read in above (or your own data).  You can start with one of the commands above or choose a different style of plot.  Whichever format you use, choose different columns to plot than above.  Try to also add a new option to the command to change the plot.  

Hint: The Plotly help pages usually contain examples for both Plotly express and Graph Object.  If you go to the [Plotly express homepage](https://plotly.com/python/plotly-express/) and click on a link (e.g., [the page for the scatter plot](https://plotly.com/python/line-and-scatter/)), you can scroll down to see Graph Object examples.

In [ ]:
# Create a plot using Plotly Graph Objects(s)

# First, create the trace


# Second, create the figure and show it



### 1.5. Show two plots side-by-side sharing the y axis.

In [ ]:
# Create the trace for the 2nd figure (similar method to above).
trace2 = go.Scatter(x = dfJoinedCOVID['START_DATE'], y = dfJoinedCOVID['2020'], 
    mode = 'markers', 
    showlegend = False,
    name = 'COVID Vaccines',
    marker = dict(
        color = 'rgba(0, 0, 0, 0.2)',
        opacity = 1,
        size = np.clip(dfJoinedCOVID['TOTAL_VACCINATIONS_PER100']/7., 5, 100),
        line = dict(
            color = 'rgba(0, 0, 0, 1)',
            width = 1
        ),
    ),
    hovertemplate = '%{text}' + 
        'Vaccine start date: %{x}<br><extra></extra>' +
        'GDP: $%{y}<br>',
    text = ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Total Vaccinations / 100 people: {}<br>'.format(x1, x2, x3) 
        for (x1, x2, x3) in zip(dfJoinedCOVID['COUNTRY'], 
            dfJoinedCOVID['Cases - cumulative total per 100000 population'], 
            dfJoinedCOVID['TOTAL_VACCINATIONS_PER100'])
    ],
    hoverlabel=dict(
        bgcolor = 'white',
    )

)

# Add trendlines
# I will use scipy.stats.linregress (and fit to the log of the GDP)
dfFit1 = dfJoinedCOVID.dropna(subset = ['TOTAL_VACCINATIONS_PER100', '2020'])
slope1, intercept1, r1, p1, se1 = scipy.stats.linregress(dfFit1['TOTAL_VACCINATIONS_PER100'], np.log10(dfFit1['2020']))
xFit1 = np.linspace(0, 300, 100)
yFit1 = 10.**(slope1*xFit1 + intercept1)
trace2F = go.Scatter(x = xFit1, y = yFit1, 
    mode = 'lines', # Set the mode the lines (rather than markers) to show a line.
    opacity = 1, 
    marker_color = 'black',
    showlegend = False,
    hoverinfo='skip' # Don't show anything on hover.  (We could show the trendline info, but I'll leave that out for now.)
)

dfFit2 = dfJoinedCOVID.dropna(subset = ['START_DATE', '2020'])
delta = (dfFit2['START_DATE'] - dfFit2['START_DATE'].min())/np.timedelta64(1,'D')
slope2, intercept2, r2, p2, se2 = scipy.stats.linregress(delta, np.log10(dfFit2['2020']))
xx2 = np.linspace(0, 500, 100)
yFit2 = 10.**(slope2*xx2 + intercept2)
xFit2 = xx2*np.timedelta64(1,'D') + dfFit2['START_DATE'].min()
trace2F = go.Scatter(x = xFit2, y = yFit2, 
    mode = 'lines', 
    opacity = 1, 
    marker_color = 'black',
    showlegend = False,
    hoverinfo='skip' 
)

# Create the figure and add the traces
# I will use Plotly's "make_subplots" method (imported above).
# Define the number of rows and columns, the column_widths, spacing, and here I will share the y axis.
# Sharing the y axis means that if you zoom/pan on one plot, the other will also zoom/pan.
fig = make_subplots(rows = 1, cols = 2, column_widths = [0.5, 0.5], horizontal_spacing = 0.01, shared_yaxes = True)

# Add the first trace and update the axes.
# Note that I specify which row and column within each of these commands.
fig.add_trace(trace1, row = 1, col = 1)
fig.add_trace(trace1F, row = 1, col = 1)
fig.update_xaxes(title = 'Total Vaccionations Per 100 People', range=[0,280], row = 1, col = 1)
fig.update_yaxes(title = 'GDP (USD)', type = 'log', row = 1, col = 1)

# Add the second trace and update the axes.
# Note that I am using numpy's datetime64 data types in order to set the axis range here
fig.add_trace(trace2, row = 1, col = 2)
fig.add_trace(trace2F, row = 1, col = 2)
fig.update_xaxes(title = 'Vaccine Start Date', range = [np.datetime64('2020-07-02'), 
                                                        np.datetime64('2021-07-01')], row = 1, col = 2)
fig.update_yaxes(type = 'log', row = 1, col = 2)

# Provide an overall title to the figure.
fig.update_layout(title_text = 'COVID-19 Vaccine Equity')

# Add annotations to tell what the symbol sizes mean.
# I will position these relative to the data domain, and therefore they will not move around when zooming and panning.
fig.add_annotation(x = 0.01, y = 0.99, row = 1, col = 1, showarrow = False,
    xref = 'x domain', yref = 'y domain',
    text = 'Symbol size indicates total COVID-19 cases.')
fig.add_annotation(x = 0.01, y = 0.99, row = 1, col = 2, showarrow = False,
    xref = 'x domain', yref = 'y domain',
    text = 'Symbol size indicates total vaccinations.')

# Show the final result
fig.show()

#### You can save the figure in html format to use on a website.

In [ ]:
fig.write_html('plotly_graph.html')

## 2. Create a plot showing COVID-19 cases and deaths vs. time for a given country.

I will also include [custom buttons](https://plotly.com/python/custom-buttons/) to toggle between various ways of viewing the data.

### 2.1. Read in the data

In [ ]:
# COVID-19 cases and deaths as a function of time for multiple countries
# dfC = pd.read_csv('data/WHO-COVID/WHO-COVID-19-global-data.csv') # in case the WHO server goes down
dfC = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')

# convert the date column to datetime objects for easier plotting and manipulation later on
dfC['Date_reported'] = pd.to_datetime(dfC['Date_reported'])
dfC

### 2.2. Choose a country, and then create the plot,

In [ ]:
country = 'United States of America'

In [ ]:
# Select only the data that is from the country.
use3 = dfC.loc[dfC['Country'] == country]

In [ ]:
# Create the trace.
# In this example I will use a bar chart.
trace3 = go.Bar(x = use3['Date_reported'], y = use3['New_cases'], 
    opacity = 1, 
    marker_color = 'black',
    showlegend = False,
    name = 'COVID Cases'
)

# Create the figure.
fig = go.Figure()

# Add the trace and update a few parameters for the axes.
fig.add_trace(trace3)
fig.update_xaxes(title = 'Date')
fig.update_yaxes(title = 'Total COVID-19 Cases')
fig.show()

#### Let's improve this plot.

- I want to take a rolling average (this is easily done with *pandas*).
- I'd prefer a filled region rather than bars.

In [ ]:
# Define the number of days to use for the rolling average.
rollingAve = 7

# Create new columns in the dataframe to contain the rolling average
dfC['New_cases_rolling'] = dfC['New_cases'].rolling(rollingAve).mean()
dfC['New_deaths_rolling'] = dfC['New_deaths'].rolling(rollingAve).mean()
dfC['Cumulative_cases_rolling'] = dfC['Cumulative_cases'].rolling(rollingAve).mean()
dfC['Cumulative_deaths_rolling'] = dfC['Cumulative_deaths'].rolling(rollingAve).mean()

# Do the selection again to update use3
use3 = dfC.loc[dfC['Country'] == country]

In [ ]:
# Create the trace, using Scatter to create lines and fill the region between the line and y=0.
trace3 = go.Scatter(x = use3['Date_reported'], y = use3['New_cases_rolling'], 
    mode = 'lines', # Set the mode the lines (rather than markers) to show a line.
    opacity = 1, 
    marker_color = 'black',
    fill = 'tozeroy',  # This will fill between the line and y=0.
    showlegend = False,
    name = 'COVID Count',
    hovertemplate = 'Date: %{x}<br>Number: %{y}<extra></extra>', #Note: the <extra></extra> removes the trace label.
)

# Create the figure.
fig = go.Figure()

# Add the trace and update a few parameters for the axes.
fig.add_trace(trace3)
fig.update_xaxes(title = 'Date')
fig.update_yaxes(title = 'Total COVID-19 Cases')
fig.show()

### *Exercise 3: Create your own plot showing COVID-19 deaths vs time.*

You can use either Plotly express or Graph Objects.  Try to pick a different country than I used above.  Also try to use a different style than I plotted above.  

In [ ]:
# Create a Plotly figure showing COVID-19 deaths vs. time




### 2.3. Add some buttons to interactively change the plot.

I want to be able to toggle between cumulative vs. total as well as cases vs. death.  We can do this with [custom buttons](https://plotly.com/python/custom-buttons/) that call a function (a callback) that will update the data in the plot.  

You can also create interactions with buttons and other "widgets" using [dash](https://plotly.com/dash/), but we won't go there in this workshop. 

In [ ]:
# Create the figure.
fig = go.Figure()

# For this scenario, I am going to add each of the 4 traces to the plot but only show one at a time
# Add traces for each column
columns = ['New_cases_rolling', 'New_deaths_rolling', 'Cumulative_cases_rolling', 'Cumulative_deaths_rolling']
for i, c in enumerate(columns):
    visible = False
    if (i == 0):
        visible = True
        
    # Create the trace, using Scatter to create lines and fill the region between the line and y=0.
    trace = go.Scatter(x = use3['Date_reported'], y = use3[c], 
        mode = 'lines', # Set the mode the lines (rather than markers) to show a line.
        opacity = 1, 
        marker_color = 'black',
        fill = 'tozeroy',  # This will fill between the line and y=0.
        showlegend = False,
        name = 'COVID Count',
        hovertemplate = 'Date: %{x}<br>Number: %{y}<extra></extra>', #Note: the <extra></extra> removes the trace label.
        visible = visible
    )
    
    # Add that trace to the figure
    fig.add_trace(trace)

# Update a few parameters for the axes.
#   Note: I added a margin to the top ('t') of the plot within fig.update_layout to make room for the buttons.
fig.update_xaxes(title = 'Date')#, range = [np.datetime64('2020-03-01'), np.datetime64('2022-01-12')])
fig.update_yaxes(title = 'COVID-19 Count')
fig.update_layout(title_text = 'COVID-19 Data Explorer : '+ country + '<br>(' + str(rollingAve) +'-day rolling average)',
    margin = dict(t = 150)
)

# Add buttons (this can also be easily done with dash, but then you can't export easily to html).
fig.update_layout(
    updatemenus = [

        # Buttons for choosing the data to plot.
        dict(
            type = 'buttons',
            direction = 'left', # This defines what orientation to include all buttons.  'left' shows them in one row.
            buttons = list([
                dict(
                    # 'args' tells the button what to do when clicked.  
                    #     In this case it will replace the y values with the output of my function
                    # 'label' is the text that will be displayed on the button
                    # 'method' is the type of action the button will take.
                    #    method = 'restyle' allows you to redefine certain preset plot styles (including the visible key).  
                    #    See  https://plotly.com/python/custom-buttons/ for different methods and their uses
                    args = [{'visible': [True, False, False, False]}], 
                    label = 'Daily Cases', 
                    method = 'restyle' 
                ),
                dict(
                    args = [{'visible': [False, True, False, False]}], 
                    label = 'Daily Deaths',
                    method = 'restyle'
                ),
                dict(
                    args = [{'visible': [False, False, True, False]}], 
                    label = 'Cumulative Cases',
                    method = 'restyle'
                ),
                dict(
                    args = [{'visible': [False, False, False, True]}], 
                    label = 'Cumulative Deaths',
                    method = 'restyle'
                )
            ]),
            showactive = True, # Highlight the active button
            # Below is for positioning
            x = 0.0, 
            xanchor = 'left',
            y = 1.13,
            yanchor = 'top'
        ),
    ]
)
    
fig.show()

### *Exercise 4: Convert the buttons into a dropdown menu.*

Using the code that I wrote above, create a plot of COVID-19 cases vs. time (for a single country) that uses a dropdown menu to choose between "Daily Cases", "Cumulative Cases", "Daily Deaths" and "Cumulative Deaths".

Hint: [This website gives examples of dropdown menus in Plotly](https://plotly.com/python/dropdowns/).

In [ ]:
# Create a plot of COVID-19 cases vs. time with a dropdown menu to change the data that is plotted.
# You can use the same trace and figure from above or create new ones.
# The code will be VERY similar to above.  In fact, you could solve this exercise with only 3 lines of code...


## 3. Put all of these plots together into one "dashboard".


### 3.1. I will put all the plotting commands (from above) into a single function.

That way I can reuse this later on for the final step (#4 below).  This is mostly copying and pasting, but with some additions that I will point out below in the comments.

In [ ]:
# I will include these parameters here in this cell for completeness (though they were already defined above).
rollingAve = 7
country = 'United States of America'

# In order to reduce the lines of code, I created a function that generates the vaccine trace, given inputs
def generateVaccineTrace(xData, yData, size, color, hovertemplate, text, hoverbg = 'white'):
    '''
        xData : the x data for the trace
        yData : the y data for the trace
        size : sizes for the data points
        color : color for the markers
        hovertemplate : the template for the tooltip
        text : the additional text to include in the tooltip
        hoverbg : optional parameter to set the background color of the tooltip (defaut is white)
    '''
    trace = go.Scatter(x = xData, y = yData, 
        mode = 'markers', 
        showlegend = False,
        name = 'COVID Vaccines',
        marker = dict(
            color = color,
            opacity = 1,
            size = size,
            line = dict(
                color = 'rgba(0, 0, 0, 1)',
                width = 1
            ),
        ),
        hovertemplate = hovertemplate,
        text = text,
        hoverlabel = dict(
            bgcolor = hoverbg,
        ),
    )
    
    return trace
    
# This is a large function that will generate the entire figure with all the subplots
def generateFigure(co):
    '''
        co : the country that we want to plot
    '''
    
    ##################################
    # First, create the traces. 
    ##################################
    
    # cases over time
    useC = dfC.loc[dfC['Country'] == co]
    traces1 = []
    columns = ['New_cases_rolling', 'New_deaths_rolling', 'Cumulative_cases_rolling', 'Cumulative_deaths_rolling']
    for i, c in enumerate(columns):
        visible = False
        if (i == 0):
            visible = True

        # Create the trace, using Scatter to create lines and fill the region between the line and y=0.
        trace = go.Scatter(x = use3['Date_reported'], y = use3[c], 
            mode = 'lines', # Set the mode the lines (rather than markers) to show a line.
            opacity = 1, 
            marker_color = 'black',
            fill = 'tozeroy',  # This will fill between the line and y=0.
            showlegend = False,
            name = 'COVID Count',
            hovertemplate = 'Date: %{x}<br>Number: %{y}<extra></extra>', #Note: the <extra></extra> removes the trace label.
            visible = visible
        )
        traces1.append(trace)
        
    
    # vaccine fraction vs. GDP (using the function that I wrote above)
    trace2 = generateVaccineTrace(dfJoinedCOVID['TOTAL_VACCINATIONS_PER100'], dfJoinedCOVID['2020'], 
        np.nan_to_num(np.clip(dfJoinedCOVID['Cases - cumulative total per 100000 population']/500., 5, 100)),
        'rgba(0, 0, 0, 0.2)',
        '%{text}' + \
            'Total Vaccinations / 100 people: %{x}<br><extra></extra>' + \
            'GDP: $%{y}<br>',
        ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Vaccine start date: {}<br>'.format(x1, x2, x3) 
            for (x1, x2, x3) in zip(dfJoinedCOVID['COUNTRY'], 
                dfJoinedCOVID['Cases - cumulative total per 100000 population'], 
                dfJoinedCOVID['START_DATE'].dt.strftime('%b %Y'))
        ],
     )
                                                                 
    # vaccine start date vs. GDP (using the function that I wrote above)
    trace3 = generateVaccineTrace(dfJoinedCOVID['START_DATE'], dfJoinedCOVID['2020'], 
        np.clip(dfJoinedCOVID['TOTAL_VACCINATIONS_PER100']/7., 5, 100),
        'rgba(0, 0, 0, 0.2)',
        '%{text}' + \
            'Vaccine start date: %{x}<br><extra></extra>' + \
            'GDP: $%{y}<br>',
        ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Total Vaccinations / 100 people: {}<br>'.format(x1, x2, x3) 
            for (x1, x2, x3) in zip(dfJoinedCOVID['COUNTRY'], 
                dfJoinedCOVID['Cases - cumulative total per 100000 population'], 
                dfJoinedCOVID['TOTAL_VACCINATIONS_PER100'])
        ],                                  
    )

    # Add trendlines
    #   This is simply copied from above
    dfFit1 = dfJoinedCOVID.dropna(subset = ['TOTAL_VACCINATIONS_PER100', '2020'])
    slope1, intercept1, r1, p1, se1 = scipy.stats.linregress(dfFit1['TOTAL_VACCINATIONS_PER100'], np.log10(dfFit1['2020']))
    xFit1 = np.linspace(0, 300, 100)
    yFit1 = 10.**(slope1*xFit1 + intercept1)
    trace2F = go.Scatter(x = xFit1, y = yFit1, 
        mode = 'lines', # Set the mode the lines (rather than markers) to show a line.
        opacity = 1, 
        marker_color = 'black',
        showlegend = False,
        hoverinfo='skip' # Don't show anything on hover.  (We could show the trendline info, but I'll leave that out for now.)
    )

    dfFit2 = dfJoinedCOVID.dropna(subset = ['START_DATE', '2020'])
    delta = (dfFit2['START_DATE'] - dfFit2['START_DATE'].min())/np.timedelta64(1,'D')
    slope2, intercept2, r2, p2, se2 = scipy.stats.linregress(delta, np.log10(dfFit2['2020']))
    xx2 = np.linspace(0, 500, 100)
    yFit2 = 10.**(slope2*xx2 + intercept2)
    xFit2 = xx2*np.timedelta64(1,'D') + dfFit2['START_DATE'].min()
    trace3F = go.Scatter(x = xFit2, y = yFit2, 
        mode = 'lines', 
        opacity = 1, 
        marker_color = 'black',
        showlegend = False,
        hoverinfo='skip' 
    )

    # Add 2 more traces for the vaccine plots to highlight the selected country (using the function that I wrote above).
    #   These are nearly identical to the 2 traces from above but using the limitted useH dataset (below) and colored red.
    useH = dfJoinedCOVID.loc[dfJoinedCOVID['COUNTRY'] == co]
    trace2H = generateVaccineTrace(useH['TOTAL_VACCINATIONS_PER100'], useH['2020'], 
        np.nan_to_num(np.clip(useH['Cases - cumulative total per 100000 population']/500., 5, 100)),
        'rgba(255, 0, 0, 1)',
        '%{text}' + \
            'Total Vaccinations / 100 people: %{x}<br><extra></extra>' + \
            'GDP: $%{y}<br>',
        ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Vaccine start date: {}<br>'.format(x1, x2, x3) 
            for (x1, x2, x3) in zip(useH['COUNTRY'], 
                useH['Cases - cumulative total per 100000 population'], 
                useH['START_DATE'].dt.strftime('%b %Y'))
        ],
        hoverbg = 'red'
    )
    trace3H = generateVaccineTrace(useH['START_DATE'], useH['2020'], 
        np.clip(useH['TOTAL_VACCINATIONS_PER100']/7., 5, 100),
        'rgba(255, 0, 0, 1)',
        '%{text}' + \
            'Vaccine start date: %{x}<br><extra></extra>' + \
            'GDP: $%{y}<br>',
        ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Total Vaccinations / 100 people: {}<br>'.format(x1, x2, x3) 
            for (x1, x2, x3) in zip(useH['COUNTRY'], 
                useH['Cases - cumulative total per 100000 population'], 
                useH['TOTAL_VACCINATIONS_PER100'])
        ],  
        hoverbg = 'red'
    )


    ##################################
    # Second, create the figure and add the traces.
    ##################################
    
    # I will create a subplot object where 
    # - the top will have 1 column and contain the cases over time,
    # - the bottom will be split in two columns for the vaccine plots,
    # - and the bottom two columns will share the y axis.
    fig = make_subplots(rows = 2, cols = 2, shared_yaxes = True,   
        column_widths = [0.5, 0.5],
        row_heights = [0.35, 0.65],
        specs = [ [{"colspan": 2}, None], [{}, {}] ], # here is where I define that the first row only has one column
        horizontal_spacing = 0.01, 
        vertical_spacing = 0.08
    )

    # Add in the traces and update the axes (specifying with row and column they below to)
    for t in traces1:
        fig.add_trace(t, row = 1, col = 1)
    fig.update_xaxes(title = 'Date')#, range = [np.datetime64('2020-03-01'), np.datetime64('2022-01-12')], row = 1, col = 1)
    fig.update_yaxes(title = 'COVID-19 Count', row = 1, col = 1, rangemode = 'nonnegative')

    fig.add_trace(trace2, row = 2, col = 1)
    fig.add_trace(trace2F, row = 2, col = 1)
    fig.add_trace(trace2H, row = 2, col = 1)
    fig.update_xaxes(title = 'Total Vaccionations Per 100 People', range=[0,280], row = 2, col = 1)
    fig.update_yaxes(title = 'GDP (USD)', type = 'log', row = 2, col = 1)

    fig.add_trace(trace3, row = 2, col = 2)
    fig.add_trace(trace3F, row = 2, col = 2)
    fig.add_trace(trace3H, row = 2, col = 2)
    fig.update_xaxes(title = 'Vaccine Start Date', range = [np.datetime64('2020-07-02'), 
                                                            np.datetime64('2021-07-01')], row = 2, col = 2)
    fig.update_yaxes(type = 'log', row = 2, col = 2)

    # Add a title and define the size and margin.
    fig.update_layout(title_text = 'COVID-19 Data Explorer : '+ co + '<br>(' + str(rollingAve) +'-day rolling average)',
        title_y = 0.97,
        height = 1000,
        width = 1000, 
        margin = dict(t = 120))

    # Add the annotations to tell what the symbol sizes mean.
    fig.add_annotation(x = 0.01, y = 0.99, row = 2, col = 1, showarrow = False,
        xref = 'x domain', yref = 'y domain',
        text = 'Symbol size indicates total COVID-19 cases.')
    fig.add_annotation(x = 0.01, y = 0.99, row = 2, col = 2, showarrow = False,
        xref = 'x domain', yref = 'y domain',
        text = 'Symbol size indicates total vaccinations.')
    

    return fig
                            
         

In [ ]:
def addButtons(fig):   
    ##################################
    # Third, add the buttons.
    ##################################
    
    # Note that here in 'args' I need to provide values for all the traces (even though only one plot will change).
    fig.update_layout(
        updatemenus = [
            dict(
                type = 'buttons',
                direction = 'left',
                buttons = list([
                    dict(
                        args = [{'visible': [True, False, False, False, True, True, True, True, True, True]}], 
                        label = 'Daily Cases', 
                        method = 'restyle' 
                    ),
                    dict(
                        args = [{'visible': [False, True, False, False, True, True, True, True, True, True]}], 
                        label = 'Daily Deaths',
                        method = 'restyle'
                    ),
                    dict(
                        args = [{'visible': [False, False, True, False, True, True, True, True, True, True]}], 
                        label = 'Cumulative Cases',
                        method = 'restyle'
                    ),
                    dict(
                        args = [{'visible': [False, False, False, True, True, True, True, True, True, True]}], 
                        label = 'Cumulative Deaths',
                        method = 'restyle'
                    )
                ]),
                showactive = True,
                x = 0.0,
                xanchor = 'left',
                y = 1.05,
                yanchor = 'top'
            ),
        ]
    )
    
    return fig

In [ ]:
# Use the functions to create the figure.
fig = generateFigure(country)
fig = addButtons(fig)
fig.show()

## 4.  *Bonus:* Add a dropdown to choose the country.

I'd like to be able to interactively choose the country to plot using a dropdown.  Plotly allows for [dropdown menus](https://plotly.com/python/dropdowns/) within a figure, in a similar method to how we added the buttons.  This code will become a bit combersome, but we'll walk through it together. 

In general, we will use the "update" mode (rather than "restyle") for the dropdown menu.  Update will allow us to change the data being plotted, using external functions.  The rest of the syntax for the dropdown menu will be very similar to the buttons.

Again, note that this could also be accomplished in [dash](https://plotly.com/dash/), but we're not going there in this workshop.  (Note that with dash you can build an app that would do this, but hosting it online is much harder than simply downloading the Plotly html file and sticking it on your website.)

In [ ]:
# Define functions that will update the data being shows in the time plot

# For the time series plot
# Since there are actually 4 traces in the time plot (only 1 visible), I will need to send 4 data sets back from each function
# This one I can just call 4 times
def updateTimePlotX(co):
    use = dfC.loc[dfC['Country'] == co]
    return use['Date_reported'] 

# There may be a smarter way to do this, but I will write 4 functions here
def updateTimePlotY1(co):
    use = dfC.loc[dfC['Country'] == co]
    return use['New_cases_rolling']

def updateTimePlotY2(co):
    use = dfC.loc[dfC['Country'] == co]
    return use['New_deaths_rolling']

def updateTimePlotY3(co):
    use = dfC.loc[dfC['Country'] == co]
    return use['Cumulative_cases_rolling']

def updateTimePlotY4(co):
    use = dfC.loc[dfC['Country'] == co]
    return use['Cumulative_deaths_rolling']

# For the vaccine data highlights
def updateVaccinePlotX1(co):    
    use = dfJoinedCOVID.loc[dfJoinedCOVID['COUNTRY'] == co]
    return use['TOTAL_VACCINATIONS_PER100']

def updateVaccinePlotX2(co):    
    use = dfJoinedCOVID.loc[dfJoinedCOVID['COUNTRY'] == co]
    return use['START_DATE']

def updateVaccinePlotY(co):
    use = dfJoinedCOVID.loc[dfJoinedCOVID['COUNTRY'] == co]
    return use['2020']

# Functions to help with the vaccine tooltip text
def getVaccineTextAll():
    return ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Vaccine start date: {}<br>'.format(x1, x2, x3) 
        for (x1, x2, x3) in zip(dfJoinedCOVID['COUNTRY'], 
            dfJoinedCOVID['Cases - cumulative total per 100000 population'], 
            dfJoinedCOVID['START_DATE'].dt.strftime('%b %Y'))
    ]

def updateVaccineHText(co):
    use = dfJoinedCOVID.loc[dfJoinedCOVID['COUNTRY'] == co]
    return ['Country: {}<br>Total COVID Cases / 100,000 people: {}<br>Total Vaccinations / 100 people: {}<br>'.format(x1, x2, x3) 
        for (x1, x2, x3) in zip(use['COUNTRY'], 
            use['Cases - cumulative total per 100000 population'], 
            use['TOTAL_VACCINATIONS_PER100'])
    ]

# Functions to help with the vaccine marker size
def getVaccineMarkersize1():
    return np.nan_to_num(np.clip(dfJoinedCOVID['Cases - cumulative total per 100000 population']/500., 5, 100))

def getVaccineMarkersize2():
    return np.clip(dfJoinedCOVID['TOTAL_VACCINATIONS_PER100']/7., 5, 100)

def getVaccineMarkersize1H(co):
    use = dfJoinedCOVID.loc[dfJoinedCOVID['COUNTRY'] == co]
    return np.nan_to_num(np.clip(use['Cases - cumulative total per 100000 population']/500., 5, 100))

def getVaccineMarkersize2H(co):
    use = dfJoinedCOVID.loc[dfJoinedCOVID['COUNTRY'] == co]
    return np.clip(use['TOTAL_VACCINATIONS_PER100']/7., 5, 100)

# Copying these fits here so that it's all in one place
slope1, intercept1, r1, p1, se1 = scipy.stats.linregress(dfFit1['TOTAL_VACCINATIONS_PER100'], np.log10(dfFit1['2020']))
xFit1 = np.linspace(0, 300, 100)
yFit1 = 10.**(slope1*xFit1 + intercept1)

dfFit2 = dfJoinedCOVID.dropna(subset = ['START_DATE', '2020'])
delta = (dfFit2['START_DATE'] - dfFit2['START_DATE'].min())/np.timedelta64(1,'D')
slope2, intercept2, r2, p2, se2 = scipy.stats.linregress(delta, np.log10(dfFit2['2020']))
xx2 = np.linspace(0, 500, 100)
yFit2 = 10.**(slope2*xx2 + intercept2)
xFit2 = xx2*np.timedelta64(1,'D') + dfFit2['START_DATE'].min()

In [ ]:
# I am going to create the dropdown list here and then add it to the figure below
# I will need to update the x and y data for the time series plot 
#    AND also update the text for the tooltips for the red circles in the bottom panel
#    AND also update the marker size for the red circles in the bottom panel
# Even though some data will not change, I will need to specify everything in this dropdown menu 

# Identify the countries to use 
# I will but The United States of America first so that it can be the default country on load (the first button)
availableCountries = dfC['Country'].unique().tolist()
availableCountries.insert(0, availableCountries.pop(availableCountries.index('United States of America'))) 

# Create the dropdown buttons
dropdown = []
for c in availableCountries:
    if (c in dfJoinedCOVID['COUNTRY'].tolist()):
        dropdown.append(dict(
            args = [{'x': [updateTimePlotX(c), updateTimePlotX(c), updateTimePlotX(c), updateTimePlotX(c), # time plot
                        dfJoinedCOVID['TOTAL_VACCINATIONS_PER100'], # full scatter plot on the left
                        xFit1, # fit line 1
                        updateVaccinePlotX1(c), # red circle in left scatter plot
                        dfJoinedCOVID['START_DATE'], # full scatter plot on the right
                        xFit2, # fit line 2
                        updateVaccinePlotX2(c) # red circle on right scatter plot
                    ],
                    'y': [updateTimePlotY1(c), updateTimePlotY2(c), updateTimePlotY3(c), updateTimePlotY4(c), 
                        dfJoinedCOVID['2020'], 
                        yFit1, 
                        updateVaccinePlotY(c), 
                        dfJoinedCOVID['2020'],
                        yFit2, 
                        updateVaccinePlotY(c)
                    ],
                    'text': ['', '', '', '',
                        getVaccineTextAll(), '', updateVaccineHText(c),
                        getVaccineTextAll(), '', updateVaccineHText(c)
                    ],
                    'marker.size': ['', '', '', '',
                        getVaccineMarkersize1(), '',getVaccineMarkersize1H(c),
                        getVaccineMarkersize2(), '',getVaccineMarkersize2H(c),
                    ]
            }],
            label = c,
            method = 'update'
        ))

In [ ]:
# A Function to add the dropdown menu and buttons
# Note: I've seen odd behavior with adding the dropdown first and then the buttons. (e.g., the dropdown turns into many buttons)
def addButtonsAndDropdown(fig):
    fig.update_layout(
        updatemenus = [

            # Copy the buttons from above
            dict(
                type = 'buttons',
                direction = 'left',
                buttons = list([
                    dict(
                        args = [{'visible': [True, False, False, False, True, True, True, True, True, True]}], 
                        label = 'Daily Cases', 
                        method = 'restyle' 
                    ),
                    dict(
                        args = [{'visible': [False, True, False, False, True, True, True, True, True, True]}], 
                        label = 'Daily Deaths',
                        method = 'restyle'
                    ),
                    dict(
                        args = [{'visible': [False, False, True, False, True, True, True, True, True, True]}], 
                        label = 'Cumulative Cases',
                        method = 'restyle'
                    ),
                    dict(
                        args = [{'visible': [False, False, False, True, True, True, True, True, True, True]}], 
                        label = 'Cumulative Deaths',
                        method = 'restyle'
                    )
                ]),
                showactive = True,
                x = 0.0,
                xanchor = 'left',
                y = 1.05,
                yanchor = 'top'
            ),

            # Add the dropdown
            dict(
                buttons = dropdown,
                    direction = 'down',
                    showactive = True,
                    x = 0.0,
                    xanchor = 'left',
                    y = 1.1,
                    yanchor = 'top'
                ),
        ]
    )
    
    return fig

In [ ]:
# I can't find a way to initialize the dropdown menu to anything other than the first in the list
# (I resorted the list to put the United States of America on top, before creating the dropdowns, so that is the default on load)
country = availableCountries[0]

# Use the function from above
fig = generateFigure(country)

# Now add in the buttons and dropdown menu 
fig = addButtonsAndDropdown(fig)

# Move the title up a bit more, and remove the country from the name
fig.update_layout(title_text = 'COVID-19 Data Explorer <br>(' + str(rollingAve) +'-day rolling average)',
        title_y = 0.97,
        margin = dict(t = 140)
)

fig.show()

In [ ]:
# You can save the plotly figure (of the given country) as an html file to use on your website.
fig.write_html('plotly_graph.html')